<a href="https://colab.research.google.com/github/jayyanar/agentic-ai-training/blob/lab-day-1/batch2/lca-langchainV1-essentials/mandatory/output/L6_memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Memory

<div style="display: flex; justify-content: flex-start; gap: 10px;">
  <img src="https://github.com/jayyanar/agentic-ai-training/blob/lab-day-1/batch2/lca-langchainV1-essentials/assets/LC_Memory_before.png?raw=1" style="width:300px; border:1px solid #ccc; border-radius:6px;">
</div>

Persisting messages, or 'agent state' between invocations of the agent.

What we're doing: Install required packages for the memory lesson in Colab.

In [3]:
!pip install -qU langchain-groq langgraph langchain-community pysqlite3-binary

What we're doing: Load the GROQ API key from Colab userdata into the environment.

In [4]:
from google.colab import userdata
import os

os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

What we're doing: Connect to the sample SQLite database and print available tables.

In [5]:
from langchain_community.utilities import SQLDatabase

# Download the database file locally
!wget -q https://github.com/jayyanar/agentic-ai-training/raw/lab-day-1/batch2/lca-langchainV1-essentials/Chinook.db

# Update the URI to the local path
db = SQLDatabase.from_uri("sqlite:///Chinook.db")

# Verify it works
print(db.get_usable_table_names())

['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


What we're doing: Define `RuntimeContext` dataclass to hold the database reference for runtime use.

In [6]:
from dataclasses import dataclass

@dataclass
class RuntimeContext:
    db: SQLDatabase

What we're doing: Define `execute_sql` tool to allow the agent to run read-only SQL queries via the runtime.

In [7]:
from langchain_core.tools import tool
from langgraph.runtime import get_runtime

@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite command and return results."""
    runtime = get_runtime(RuntimeContext)
    db = runtime.context.db

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

What we're doing: Set `SYSTEM_PROMPT` rules to constrain agent SQL behavior and ensure read-only queries.

In [8]:
SYSTEM_PROMPT = """You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
"""

What we're doing: Initialize the Groq model and create an agent with `execute_sql` and runtime context.

In [9]:
from langchain.agents import create_agent
from langchain_groq import ChatGroq

# Initialize the Groq model
llm = ChatGroq(
    model="openai/gpt-oss-120b",
    temperature=0,
    max_retries=2,
)

agent = create_agent(
    #model="openai:gpt-5",
    model=llm,
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    context_schema=RuntimeContext,
)

## Repeated Queries

What we're doing: Stream a repeated query scenario to observe agent behavior without memory enabled.

In [10]:
question = "This is Frank Harris, What was the total on my last invoice?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
    context=RuntimeContext(db=db),
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

This is Frank Harris, What was the total on my last invoice?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (fc_8de94ec3-af01-4c53-8d73-3e690957c1d6)
 Call ID: fc_8de94ec3-af01-4c53-8d73-3e690957c1d6
  Args:
    query: SELECT name FROM sqlite_master WHERE type='table';
================================= Tool Message =================================
Name: execute_sql

[('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',), ('Invoice',), ('InvoiceLine',), ('MediaType',), ('Playlist',), ('PlaylistTrack',), ('Track',)]
================================== Ai Message ==================================
Tool Calls:
  execute_sql (fc_6f7a0e9e-3d9e-4929-b8c7-5c9922acdbe4)
 Call ID: fc_6f7a0e9e-3d9e-4929-b8c7-5c9922acdbe4
  Args:
    query: PRAGMA table_info('Customer');
================================= Tool Message =============================

What we're doing: Follow up query without memory to show that prior context isn't retained across invocations.

In [11]:
question = "What were the titles?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What were the titles?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (fc_926868ae-2a5c-47c9-a0f4-3e06459d708c)
 Call ID: fc_926868ae-2a5c-47c9-a0f4-3e06459d708c
  Args:
    query: SELECT name FROM sqlite_master WHERE type='table' LIMIT 5;
================================= Tool Message =================================
Name: execute_sql

[('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',)]
================================== Ai Message ==================================
Tool Calls:
  execute_sql (fc_a0f0ac86-e445-401c-ba93-cb4985680cc0)
 Call ID: fc_a0f0ac86-e445-401c-ba93-cb4985680cc0
  Args:
    query: PRAGMA table_info('Track');
================================= Tool Message =================================
Name: execute_sql

[(0, 'TrackId', 'INTEGER', 1, None, 1), (1, 'Name', 'NVARCHAR(200)', 1, None, 0), (2, 'AlbumId', 'INTEGER

## Add memory

What we're doing: Import an in-memory checkpointer to enable simple agent memory persistence.

In [12]:
from langgraph.checkpoint.memory import InMemorySaver

What we're doing: Re-create the agent with an in-memory checkpointer so it can remember prior interactions.

In [13]:
from langchain.agents import create_agent
from langchain_core.messages import SystemMessage

agent = create_agent(
    #model="openai:gpt-5",
    model=llm,
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    context_schema=RuntimeContext,
    checkpointer=InMemorySaver(),
)

What we're doing: Run the same invoice query with memory enabled (thread_id=1) to persist context between calls.

In [14]:
question = "This is Frank Harris, What was the total on my last invoice?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

This is Frank Harris, What was the total on my last invoice?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (fc_b5a94d38-b501-4ac1-8eba-1b80c5fe1349)
 Call ID: fc_b5a94d38-b501-4ac1-8eba-1b80c5fe1349
  Args:
    query: SELECT name FROM sqlite_master WHERE type='table';
================================= Tool Message =================================
Name: execute_sql

[('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',), ('Invoice',), ('InvoiceLine',), ('MediaType',), ('Playlist',), ('PlaylistTrack',), ('Track',)]
================================== Ai Message ==================================
Tool Calls:
  execute_sql (fc_4183a698-1f42-44fb-9d55-0393173c13dc)
 Call ID: fc_4183a698-1f42-44fb-9d55-0393173c13dc
  Args:
    query: SELECT CustomerId FROM Customer WHERE FirstName = 'Frank' AND LastName = 'Harris';
========================

What we're doing: Ask a follow-up question using the same memory `thread_id` to check recall of details like titles.

In [15]:
question = "What were the titles?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

What were the titles?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (fc_450b7e94-3629-4572-85ae-2456128f3836)
 Call ID: fc_450b7e94-3629-4572-85ae-2456128f3836
  Args:
    query: SELECT t.Name FROM InvoiceLine il JOIN Track t ON il.TrackId = t.TrackId WHERE il.InvoiceId = 374;
================================= Tool Message =================================
Name: execute_sql

[('Holier Than Thou',), ('Through The Never',), ('My Friend Of Misery',), ('The Wait',), ('Blitzkrieg',), ('So What',)]
================================== Ai Message ==================================

Here are the track titles on that invoice:

1. Holier Than Thou  
2. Through The Never  
3. My Friend Of Misery  
4. The Wait  
5. Blitzkrieg  
6. **So What** (the last track)


## Try your own queries
Now that there is memory, check the agents recall!

What we're doing: Placeholder for you to enter your own question and test agent memory behavior.

In [16]:
question = "Your Question Here?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

Your Question Here?
================================== Ai Message ==================================

Sure thing! What else would you like to explore about your purchases—perhaps the total amount per track, details from earlier invoices, or something else entirely?


What we're doing: Example follow-up that asks the agent to show full results using the same memory thread.

In [17]:
question = "yes i want to see the full list"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

yes i want to see the full list
================================== Ai Message ==================================
Tool Calls:
  execute_sql (fc_9c65ba4c-d2cb-494d-bc4d-ca98f4218e71)
 Call ID: fc_9c65ba4c-d2cb-494d-bc4d-ca98f4218e71
  Args:
    query: SELECT COUNT(*) FROM InvoiceLine il JOIN Invoice i ON il.InvoiceId = i.InvoiceId WHERE i.CustomerId = 16;
================================= Tool Message =================================
Name: execute_sql

[(38,)]
================================== Ai Message ==================================
Tool Calls:
  execute_sql (fc_4f00563a-6dcd-4511-95fa-1305127ccec9)
 Call ID: fc_4f00563a-6dcd-4511-95fa-1305127ccec9
  Args:
    query: SELECT i.InvoiceId, i.InvoiceDate, t.Name AS TrackTitle FROM Invoice i JOIN InvoiceLine il ON i.InvoiceId = il.InvoiceId JOIN Track t ON il.TrackId = t.TrackId WHERE i.CustomerId = 16 ORDER BY i.InvoiceDate, i.InvoiceId;
===============

(End of notebook)